<a href="https://colab.research.google.com/github/imrahul1/MIDAS-Internship/blob/main/Copy_of_MIDAS_Task_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary modules and mounting the drive

In [ ]:
import pandas as pd
import re

In [ ]:
import string

import nltk
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import string

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Copy of flipkart_com-ecommerce_sample').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df = pd.DataFrame.from_records(rows)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
1,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999,379,"[""http://img5a.flixcart.com/image/short/u/4/a/...",FALSE,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
2,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157,22646,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FALSE,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
3,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999,499,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",FALSE,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
4,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699,267,"[""http://img5a.flixcart.com/image/short/6/2/h/...",FALSE,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."


Performing EDA on the dataframe

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20001 non-null  object
 1   1       20001 non-null  object
 2   2       20001 non-null  object
 3   3       20001 non-null  object
 4   4       20001 non-null  object
 5   5       20001 non-null  object
 6   6       20001 non-null  object
 7   7       20001 non-null  object
 8   8       20001 non-null  object
 9   9       20001 non-null  object
 10  10      20001 non-null  object
 11  11      20001 non-null  object
 12  12      20001 non-null  object
 13  13      20001 non-null  object
 14  14      20001 non-null  object
dtypes: object(15)
memory usage: 2.3+ MB


In [ ]:
new_header = df.iloc[0] 

df = df[1:] 

df.columns = new_header

In [ ]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
1,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999,379,"[""http://img5a.flixcart.com/image/short/u/4/a/...",FALSE,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
2,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157,22646,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FALSE,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
3,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999,499,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",FALSE,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
4,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699,267,"[""http://img5a.flixcart.com/image/short/6/2/h/...",FALSE,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
5,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220,210,"[""http://img5a.flixcart.com/image/pet-shampoo/...",FALSE,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
df.product_specifications.values[:1]

array(['{"product_specification"=>[{"key"=>"Number of Contents in Sales Package", "value"=>"Pack of 3"}, {"key"=>"Fabric", "value"=>"Cotton Lycra"}, {"key"=>"Type", "value"=>"Cycling Shorts"}, {"key"=>"Pattern", "value"=>"Solid"}, {"key"=>"Ideal For", "value"=>"Women\'s"}, {"value"=>"Gentle Machine Wash in Lukewarm Water, Do Not Bleach"}, {"key"=>"Style Code", "value"=>"ALTHT_3P_21"}, {"value"=>"3 shorts"}]}'],
      dtype=object)

In [ ]:
df.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object', name=0)

In [ ]:
df[['product_name', 'product_category_tree', 'description']]

,product_name,product_category_tree,description
1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...
3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...
4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...
...,...,...,...
19996,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy WallDesign Small Vinyl Sticker for Rs.730 ...
19997,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy Wallmantra Large Vinyl Stickers Sticker fo...
19998,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy Elite Collection Medium Acrylic Sticker fo...
19999,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy Elite Collection Medium Acrylic Sticker fo...


In [ ]:
df.product_category_tree.value_counts()

["Jewellery >> Necklaces & Chains >> Necklaces"]                                                                                                                         1567
["Jewellery >> Rings"]                                                                                                                                                    710
["Automotive >> Accessories & Spare parts >> Car Interior & Exterior >> Car Interior >> Car Mats"]                                                                        522
["Jewellery >> Bangles, Bracelets & Armlets >> Bangles"]                                                                                                                  430
["Tools & Hardware >> Tools >> Gardening Tools >> Plant Containers & Sets >> Plant Container Sets"]                                                                       332
                                                                                                                                  

In [ ]:
df.product_category_tree[:1].values

array(['["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'],
      dtype=object)

In [ ]:
text = df.product_category_tree[:1].values[0]

In [ ]:
text

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

<h1>Preprocessing</h1>

Extracting the parent category from the product_category_tree column

In [ ]:
def extract(word):
  result = re.split(">>",word)
  final = result[0][2:]
  return final

In [ ]:
df2 = df[['product_name', 'product_category_tree', 'description']]

In [ ]:
df2.head()

,product_name,product_category_tree,description
1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...
3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...
4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...


In [ ]:
df2['category'] = df2['product_category_tree'].map(extract)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Now we have the extracted the parent category

In [ ]:
df2.head()

,product_name,product_category_tree,description,category
1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing
2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,Furniture
3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,Footwear
4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing
5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Pet Supplies


The below categories are mistakes. We will remove them as they are not huge in number. They can be used for testing later one though.

In [ ]:
df2.category.value_counts()[-238:]

Clovia Women's Full Coverage Bra"]             9
Vishudh Printed Women's Straight Kurta"]       8
Lilliput Top Baby Girl's  Combo"]              8
Olvin Aviator Sunglasses"]                     7
Clovia Women's T-Shirt Bra"]                   6
                                              ..
Laser X Checkered Men's Boxer (Pack of 4)"]    1
Pazel Slim Fit Men's Jeans"]                   1
ABEEZ Boys, Men, Girls (Black, Pack of 1)"]    1
Impala Alloy Cufflink (White)"]                1
Miss Wow Slim Fit Women's Blue Jeans"]         1
Name: category, Length: 238, dtype: int64

In [ ]:
eliminate = df2.category.value_counts()[-238:]

In [ ]:
eliminate

Clovia Women's Full Coverage Bra"]             9
Vishudh Printed Women's Straight Kurta"]       8
Lilliput Top Baby Girl's  Combo"]              8
Olvin Aviator Sunglasses"]                     7
Clovia Women's T-Shirt Bra"]                   6
                                              ..
Laser X Checkered Men's Boxer (Pack of 4)"]    1
Pazel Slim Fit Men's Jeans"]                   1
ABEEZ Boys, Men, Girls (Black, Pack of 1)"]    1
Impala Alloy Cufflink (White)"]                1
Miss Wow Slim Fit Women's Blue Jeans"]         1
Name: category, Length: 238, dtype: int64

In [ ]:
eliminate = pd.DataFrame(eliminate)

In [ ]:
eliminate.head()

,category
"Clovia Women's Full Coverage Bra""]",9
"Vishudh Printed Women's Straight Kurta""]",8
"Lilliput Top Baby Girl's Combo""]",8
"Olvin Aviator Sunglasses""]",7
"Clovia Women's T-Shirt Bra""]",6


In [ ]:
eliminate.rename(columns={'category':'frequency'}, inplace=True)

In [ ]:
eliminate.head()

,frequency
"Clovia Women's Full Coverage Bra""]",9
"Vishudh Printed Women's Straight Kurta""]",8
"Lilliput Top Baby Girl's Combo""]",8
"Olvin Aviator Sunglasses""]",7
"Clovia Women's T-Shirt Bra""]",6


In [ ]:
eliminate['category'] = eliminate.index

In [ ]:
eliminate

,frequency,category
"Clovia Women's Full Coverage Bra""]",9,"Clovia Women's Full Coverage Bra""]"
"Vishudh Printed Women's Straight Kurta""]",8,"Vishudh Printed Women's Straight Kurta""]"
"Lilliput Top Baby Girl's Combo""]",8,"Lilliput Top Baby Girl's Combo""]"
"Olvin Aviator Sunglasses""]",7,"Olvin Aviator Sunglasses""]"
"Clovia Women's T-Shirt Bra""]",6,"Clovia Women's T-Shirt Bra""]"
...,...,...
"Laser X Checkered Men's Boxer (Pack of 4)""]",1,"Laser X Checkered Men's Boxer (Pack of 4)""]"
"Pazel Slim Fit Men's Jeans""]",1,"Pazel Slim Fit Men's Jeans""]"
"ABEEZ Boys, Men, Girls (Black, Pack of 1)""]",1,"ABEEZ Boys, Men, Girls (Black, Pack of 1)""]"
"Impala Alloy Cufflink (White)""]",1,"Impala Alloy Cufflink (White)""]"


Here we have removed the incorrect category values from the dataframe

In [ ]:
for i in eliminate.category.values:
  if i in df2.category.values:
    df2 = df2[df2['category']!= i]
  else:
    continue

In [ ]:
df2.category.values

array(['Clothing ', 'Furniture ', 'Footwear ', ..., 'Baby Care ',
       'Baby Care ', 'Baby Care '], dtype=object)

In [ ]:
df2.category.value_counts()

Clothing                              6198
Jewellery                             3531
Footwear                              1227
Mobiles & Accessories                 1099
Automotive                            1012
Home Decor & Festive Needs             929
Beauty and Personal Care               710
Home Furnishing                        700
Kitchen & Dining                       647
Computers                              578
Watches                                530
Baby Care                              483
Tools & Hardware                       391
Toys & School Supplies                 330
Pens & Stationery                      313
Bags, Wallets & Belts                  265
Furniture                              180
Sports & Fitness                       166
Cameras & Accessories                   82
Home Improvement                        81
Health & Personal Care Appliances       43
Gaming                                  35
Sunglasses                              35
Pet Supplie

In [ ]:
df.product_category_tree.value_counts

<bound method IndexOpsMixin.value_counts of 1        ["Clothing >> Women's Clothing >> Lingerie, Sl...
2        ["Furniture >> Living Room Furniture >> Sofa B...
3        ["Footwear >> Women's Footwear >> Ballerinas >...
4        ["Clothing >> Women's Clothing >> Lingerie, Sl...
5        ["Pet Supplies >> Grooming >> Skin & Coat Care...
                               ...                        
19996    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19997    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19998    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19999    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
20000    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
Name: product_category_tree, Length: 20000, dtype: object>

In [ ]:
df.product_category_tree.value_counts().sum()

20000

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
def remove_punctuation_and_stopwords(mess):
    
    no_punct = [ch for ch in mess if ch not in string.punctuation]
    no_punct = "".join(no_punct).split()
    
    no_punct_no_stopw = [word.lower() for word in no_punct if word.lower() not in stopwords.words("english")]
    msg = " ".join([stemmer.stem(word) for word in no_punct_no_stopw])
        
    return msg

In [ ]:
df2['description'] = df2['description'].apply(remove_punctuation_and_stopwords)

In [ ]:
df2.head()

,product_name,product_category_tree,description,category
1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",key featur alisha solid women cycl short cotto...,Clothing
2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",fabhomedecor fabric doubl sofa bed finish colo...,Furniture
3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",key featur aw belli sandal wedg heel casualsaw...,Footwear
4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",key featur alisha solid women cycl short cotto...,Clothing
5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",specif sicon purpos arnica dog shampoo 500 ml ...,Pet Supplies


In [ ]:
categories = df2.category.value_counts()

Vectorization

In [ ]:
count = CountVectorizer(remove_punctuation_and_stopwords).fit(df2['description'])
print(len(count.vocabulary_))

27562


In [ ]:
count_tran = count.transform(df2['description'])

Providing weights to frequently occuring words.

In [ ]:
tfidf =TfidfTransformer().fit(count_tran)

In [ ]:
data_tfidf = tfidf.transform(count_tran)

<h1> Modelling </h1>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_tfidf, df2['category'], test_size=0.3, random_state=5)

In [ ]:
def classification(clf):
    fit = clf.fit(X_train, y_train)
    predict = fit.predict(X_test)
    print('accuracy:', accuracy_score(y_test, predict))
    print(classification_report(y_test, predict))

In [ ]:
classification(SVC())

accuracy: 0.9654178674351584
                                    precision    recall  f1-score   support

                       Automotive        0.97      0.99      0.98       300
                        Baby Care        0.94      0.81      0.87       140
            Bags, Wallets & Belts        0.96      0.88      0.92        84
         Beauty and Personal Care        0.98      0.96      0.97       225
            Cameras & Accessories        1.00      0.88      0.93        24
                         Clothing        0.96      1.00      0.98      1845
                        Computers        0.92      0.94      0.93       177
                          Eyewear        1.00      1.00      1.00         1
                         Footwear        0.99      0.99      0.99       373
                        Furniture        0.93      0.92      0.93        62
                           Gaming        0.80      0.50      0.62         8
Health & Personal Care Appliances        1.00      0.70   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classification(MultinomialNB())

accuracy: 0.797423292083404
                                    precision    recall  f1-score   support

                       Automotive        0.94      0.94      0.94       300
                        Baby Care        0.98      0.31      0.47       140
            Bags, Wallets & Belts        1.00      0.02      0.05        84
         Beauty and Personal Care        1.00      0.30      0.46       225
            Cameras & Accessories        0.00      0.00      0.00        24
                         Clothing        0.78      1.00      0.87      1845
                        Computers        1.00      0.49      0.65       177
                          Eyewear        0.00      0.00      0.00         1
                         Footwear        1.00      0.76      0.86       373
                        Furniture        1.00      0.03      0.06        62
                           Gaming        0.00      0.00      0.00         8
Health & Personal Care Appliances        0.00      0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classification(KNeighborsClassifier(n_neighbors=49))

accuracy: 0.917782675029666
                                    precision    recall  f1-score   support

                       Automotive        0.95      0.95      0.95       300
                        Baby Care        0.77      0.59      0.67       140
            Bags, Wallets & Belts        0.89      0.65      0.75        84
         Beauty and Personal Care        0.90      0.88      0.89       225
            Cameras & Accessories        1.00      0.75      0.86        24
                         Clothing        0.95      1.00      0.97      1845
                        Computers        0.83      0.80      0.81       177
                          Eyewear        0.00      0.00      0.00         1
                         Footwear        0.85      0.98      0.91       373
                        Furniture        0.76      0.90      0.82        62
                           Gaming        0.38      0.38      0.38         8
Health & Personal Care Appliances        1.00      0.20    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classification(DecisionTreeClassifier(min_samples_split=7, random_state=111))

accuracy: 0.9442278352263096
                                    precision    recall  f1-score   support

                       Automotive        0.96      0.95      0.95       300
                        Baby Care        0.85      0.83      0.84       140
            Bags, Wallets & Belts        0.92      0.82      0.87        84
         Beauty and Personal Care        0.95      0.92      0.93       225
            Cameras & Accessories        0.96      0.92      0.94        24
                         Clothing        0.98      0.98      0.98      1845
                        Computers        0.88      0.94      0.91       177
                          Eyewear        0.50      1.00      0.67         1
                         Footwear        0.98      0.97      0.97       373
                        Furniture        0.76      0.90      0.82        62
                           Gaming        0.62      0.62      0.62         8
Health & Personal Care Appliances        0.78      0.70   

In [ ]:
classification(RandomForestClassifier(n_estimators=31, random_state=111))

accuracy: 0.9566028140362773
                                    precision    recall  f1-score   support

                       Automotive        0.95      1.00      0.97       300
                        Baby Care        0.99      0.64      0.78       140
            Bags, Wallets & Belts        1.00      0.76      0.86        84
         Beauty and Personal Care        0.98      0.92      0.95       225
            Cameras & Accessories        1.00      0.79      0.88        24
                         Clothing        0.95      1.00      0.97      1845
                        Computers        0.96      0.90      0.93       177
                          Eyewear        1.00      1.00      1.00         1
                         Footwear        0.99      0.96      0.97       373
                        Furniture        0.93      0.90      0.92        62
                           Gaming        0.80      0.50      0.62         8
Health & Personal Care Appliances        0.89      0.80   